In [3]:
!pip install nltk
!pip install sklearn

In [4]:
import nltk, string, numpy , math

In [5]:
d1 = "plot: two teen couples go to a church party, drink and then drive."
d2 = "films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before . "
d3 = "every now and then a movie comes along from a suspect studio , with every indication that it will be a stinker , and to everybody's surprise ( perhaps even the studio ) the film becomes a critical darling . "
d4 = "damn that y2k bug ."

In [6]:
documents = [d1, d2, d3, d4]

# PreProcessing

In [7]:
# Normalize by stemming
# ---------------------------------
nltk.download('punkt') # first-time use only
stemmer = nltk.stem.porter.PorterStemmer()
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# Normalize by lemmatization
# -------------------------------------
# nltk.download('wordnet') # first-time use only
lemmer = nltk.stem.WordNetLemmatizer()
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [8]:
def StemTokens(tokens):
    return [stemmer.stem(token) for token in tokens]
def StemNormalize(text):
    return StemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

#-----------------------

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

LemVectorizer = CountVectorizer(tokenizer=LemNormalize, stop_words='english')
LemVectorizer.fit_transform(documents)
print(LemVectorizer.vocabulary_)

{'plot': 27, 'teen': 37, 'couple': 9, 'church': 6, 'party': 25, 'drink': 14, 'drive': 15, 'film': 17, 'adapted': 0, 'comic': 8, 'book': 3, 'plenty': 26, 'success': 32, 'theyre': 38, 'superheroes': 33, 'batman': 2, 'superman': 34, 'spawn': 29, 'geared': 18, 'kid': 22, 'casper': 5, 'arthouse': 1, 'crowd': 11, 'ghost': 19, 'world': 39, 'really': 28, 'like': 23, 'hell': 20, 'movie': 24, 'come': 7, 'suspect': 36, 'studio': 31, 'indication': 21, 'stinker': 30, 'everybodys': 16, 'surprise': 35, 'critical': 10, 'darling': 13, 'damn': 12, 'y2k': 40, 'bug': 4}


/home/israel/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


# using TfidfVectorizer 

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
def cos_similarity(textlist):
    tfidf = TfidfVec.fit_transform(textlist)
    return (tfidf * tfidf.T).toarray()
print(cos_similarity(documents))

[[1.         0.         0.         0.        ]
 [0.         1.         0.03264186 0.        ]
 [0.         0.03264186 1.         0.        ]
 [0.         0.         0.         1.        ]]


# Exploring Tfidf

In [30]:
tf_matrix = LemVectorizer.transform(documents).toarray()
tfidfTran = TfidfTransformer(norm="l2")
tfidfTran.fit(tf_matrix)
print(tfidfTran.idf_)

[1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.51082562
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073]


In [31]:
def idf(n,df):
    result = math.log((n+1.0)/(df+1.0)) + 1
    return result
print("The idf for terms that appear in one document: " + str(idf(4,1)))
print("The idf for terms that appear in two documents: " + str(idf(4,2)))
print("The idf for terms that appear in three documents: " + str(idf(4,3)))
print("The idf for terms that appear in four documents: " + str(idf(4,4)))

The idf for terms that appear in one document: 1.916290731874155
The idf for terms that appear in two documents: 1.5108256237659907
The idf for terms that appear in three documents: 1.2231435513142097
The idf for terms that appear in four documents: 1.0


In [32]:
tfidf_matrix = tfidfTran.transform(tf_matrix)
cos_similarity_matrix = (tfidf_matrix * tfidf_matrix.T).toarray()
print(cos_similarity_matrix)

[[1.         0.         0.         0.        ]
 [0.         1.         0.03264186 0.        ]
 [0.         0.03264186 1.         0.        ]
 [0.         0.         0.         1.        ]]
